# SQLAlchemy简单教程

通过一个简单示例，完成连接数据库、建表，并进行增删改查操作

环境准备：

```shell
pip install sqlalchemy
```

## 连接数据库

create_engine的参数有很多，我列一些比较常用的：

- echo=False -- 如果为真，引擎将记录所有语句以及 repr() 其参数列表的默认日志处理程序。
- enable_from_linting -- 默认为True。如果发现给定的SELECT语句与将导致笛卡尔积的元素取消链接，则将发出警告。
- encoding -- 默认为 utf-8
- future -- 使用2.0样式
- hide_parameters -- 布尔值，当设置为True时，SQL语句参数将不会显示在信息日志中，也不会格式化为 StatementError 对象。
- listeners -- 一个或多个列表 PoolListener 将接收连接池事件的对象。
- logging_name -- 字符串标识符，默认为对象id的十六进制字符串。
- max_identifier_length -- 整数；重写方言确定的最大标识符长度。
- max_overflow=10 -- 允许在连接池中“溢出”的连接数，即可以在池大小设置（默认为5）之上或之外打开的连接数。
- pool_size=5 -- 在连接池中保持打开的连接数
- plugins -- 要加载的插件名称的字符串列表。

In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///test.db', echo=True)

## 创建数据表

声明映射，就是我们在Python中创建的一个类，对应着数据库中的一张表，类的每个属性，就是这个表的字段名。
这种的类对应于数据库中表的类，就称为映射类，我们要创建一个映射类，是基于基类定义的，每个映射类都要继承这个基类 declarative_base()。

In [2]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

    def __repr__(self):
        return f"<User(id={self.id}, name={self.name}, age={self.age})>"

Base.metadata.create_all(engine)


2025-02-21 16:03:21,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-21 16:03:21,097 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-02-21 16:03:21,098 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-21 16:03:21,099 INFO sqlalchemy.engine.Engine COMMIT


## 创建sql会话

In [14]:
from sqlalchemy.orm import sessionmaker
from contextlib import contextmanager


Session = sessionmaker(bind=engine)
session = Session()

@contextmanager
def get_db():
    db = Session()
    try:
        yield db
    finally:
        db.close()


## 插入数据

In [4]:

user_data = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 32},
    {"name": "Jmey", "age": 13},
    {"name": "Tank", "age": 9},
    {"name": "Sandy", "age": 87},
    {"name": "Neky", "age": 98},
]
for user in user_data:
    u = User(**user)
    session.add(u)

session.commit()

2025-02-21 16:03:26,754 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-21 16:03:26,758 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING id
2025-02-21 16:03:26,759 INFO sqlalchemy.engine.Engine [generated in 0.00022s (insertmanyvalues) 1/6 (ordered; batch not supported)] ('Alice', 30)
2025-02-21 16:03:26,761 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING id
2025-02-21 16:03:26,762 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/6 (ordered; batch not supported)] ('Bob', 32)
2025-02-21 16:03:26,763 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING id
2025-02-21 16:03:26,764 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/6 (ordered; batch not supported)] ('Jmey', 13)
2025-02-21 16:03:26,767 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING id
2025-02-21 16:03:26,768 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/6 (ordered; batch not su

## 查询所有数据

- query.filter() 过滤
- query.filter_by() 根据关键字过滤
- query.all() 返回列表
- query.first() 返回第一个元素
- query.one() 有且只有一个元素时才正确返回
- query.one_or_none()，类似one，但如果没有找到结果，则不会引发错误
- query.scalar()，调用one方法，并在成功时返回行的第一列
- query.count() 计数
- query.order_by() 排序
- query.join() 连接查询

In [15]:
with get_db() as s:
    users = s.query(User).all()

for user in users:
    print(user)

2025-02-21 16:41:41,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-21 16:41:41,242 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users
2025-02-21 16:41:41,244 INFO sqlalchemy.engine.Engine [cached since 2288s ago] ()
2025-02-21 16:41:41,255 INFO sqlalchemy.engine.Engine ROLLBACK
<User(id=1, name=Alice, age=35)>
<User(id=2, name=Alice, age=30)>
<User(id=3, name=Bob, age=32)>
<User(id=4, name=Jmey, age=13)>
<User(id=6, name=Sandy, age=87)>
<User(id=7, name=Neky, age=98)>
<User(id=8, name=Alice, age=30)>
<User(id=9, name=Bob, age=32)>
<User(id=10, name=Jmey, age=13)>
<User(id=12, name=Sandy, age=87)>
<User(id=13, name=Neky, age=98)>
<User(id=14, name=Alice, age=30)>
<User(id=15, name=Bob, age=32)>
<User(id=16, name=Jmey, age=13)>
<User(id=18, name=Sandy, age=87)>
<User(id=19, name=Neky, age=98)>
<User(id=20, name=Alice, age=30)>
<User(id=21, name=Bob, age=32)>
<User(id=22, name=Jmey, age=13)>
<User(id=24, name

## 查询单条数据

In [16]:
user = session.query(User).filter_by(name='Alice').first()
print(user)

2025-02-21 16:41:52,099 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-21 16:41:52,103 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2025-02-21 16:41:52,104 INFO sqlalchemy.engine.Engine [cached since 2295s ago] ('Alice', 1, 0)
<User(id=1, name=Alice, age=35)>


## 修改数据

In [9]:
user = session.query(User).filter_by(name='Alice').first()
if user:
    user.age = 35
    session.commit()

2025-02-21 16:03:58,045 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2025-02-21 16:03:58,047 INFO sqlalchemy.engine.Engine [cached since 20.69s ago] ('Alice', 1, 0)
2025-02-21 16:03:58,053 INFO sqlalchemy.engine.Engine UPDATE users SET age=? WHERE users.id = ?
2025-02-21 16:03:58,053 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (35, 1)
2025-02-21 16:03:58,055 INFO sqlalchemy.engine.Engine COMMIT


## 删除数据

In [11]:
user = session.query(User).filter_by(name='Tank').first()
if user:
    session.delete(user)
    session.commit()

2025-02-21 16:04:07,249 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2025-02-21 16:04:07,251 INFO sqlalchemy.engine.Engine [cached since 29.9s ago] ('Tank', 1, 0)
2025-02-21 16:04:07,254 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2025-02-21 16:04:07,255 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (23,)
2025-02-21 16:04:07,257 INFO sqlalchemy.engine.Engine COMMIT
